## Inserting Rows with Core

When using _Core_, a `SQL INSERT statement` is generated using the `insert()` function - this function generates a new instance of `Insert` which represents an `INSERT statement` in __SQL__, that _adds new data into a table_.

> __ORM Readers__ - The way that rows are `INSERT`ed into the database from an ORM perspective makes use of _object-centric APIs_ on the `Session` object known as the _unit of work process_, and is fairly __different from the Core-only approach__ described here. The more ORM-focused sections later starting at Inserting Rows with the ORM subsequent to the Expression Language sections introduce this.

#### Primary setup

In [4]:
from sqlalchemy import (
    MetaData, Table, Column, Integer, String,
    ForeignKey, create_engine, insert
)

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [3]:
metadata_obj = MetaData()

user_table = Table(
    "user_account", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

metadata_obj.create_all(engine)

2022-09-21 06:46:58,469 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-21 06:46:58,470 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-21 06:46:58,472 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-21 06:46:58,474 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-09-21 06:46:58,476 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-21 06:46:58,478 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-21 06:46:58,479 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-21 06:46:58,481 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-09-21 06:46:58,482 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-21 06:46:58,485 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-21 06:46:58,486 INFO sqlalchemy.engine.Engine [no key 0.00132s] ()
2022-09-21 06:46:58,489 INFO sqlalchemy.engine.Engine 
C

#### The `insert()` SQL Expression Construct

A simple example of `Insert` illustrating the `target table` and the `VALUES clause` at once:

In [7]:
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


The above _stmt variable_ is an instance of `Insert`. Most SQL expressions can be stringified in place as a means to see the general form of what's being produced.

The _stringified_ form is created by producing a `Compiled` form of the object which includes a __database-specific string SQL representation__ of the statement; we can acquire this object directly using the `ClauseElement.compile()` method.

Our `Insert` construct is an example of a `"parameterized" construct`; to view the `name` and `fullname` _bound parameters_, these are available from the `Compiled` construct as well.

In [8]:
compiled = stmt.compile()
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

#### Executing the Statement

Invoking the statement we can _INSERT a row into `user_table`_. The `INSERT SQL` as well as the _bundled parameters_ can be seen in the `SQL logging`.

In [9]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2022-09-21 07:08:43,119 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-21 07:08:43,120 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-21 07:08:43,121 INFO sqlalchemy.engine.Engine [generated in 0.00244s] ('spongebob', 'Spongebob Squarepants')
2022-09-21 07:08:43,123 INFO sqlalchemy.engine.Engine COMMIT


In its simple form above, _the `INSERT statement` does not return any rows_, and if `only a single row` is inserted, it will _usually include_ the ability to _return information about column-level default values_ that were generated during the `INSERT` of that row, most commonly an integer primary key value. In the above case the first row in a SQLite database will normally return 1 for the first integer primary key value, which we can acquire using the `CursorResult.inserted_primary_key` accessor.

In [10]:
result.inserted_primary_key

(1,)

> __CursorResult.inserted_primary_key__ returns a `tuple` because _a primary key may contain multiple columns_. This is known as a __composite primary key__. The `CursorResult.inserted_primary_key` is intended to always contain the _complete primary key_ of the record _just inserted_, not just a `cursor.lastrowid` kind of value, and is also intended to be populated regardless of whether or not `autoincrement` were used, hence _to express a complete primary key it's a tuple_.

> From version 1.4.8, the tuple returned by `CursorResult.inserted_primary_key` is now a __named tuple__ fulfilled by returning it as a `Row` object.